# Análise Estrutural & Elementos Finitos

In [1]:
import math
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.tri as mtri
import meshio
from scipy.spatial import Delaunay
import scipy as sp
import pandas as pd
import os                # Library used for system interaction (such as listing files in a directory,...)
import itertools as it   #biblioteca com funções de permutação a serem usadas na interação da matriz de conectividade com a global

## Funções de forma (ou interpolação)

In [3]:
#1D----------------------------------------------
##1ª ordem:
#Nh1 = [Nh11, Nh12, Nh13, Nh14]

def Nh11(x,l):
    return 2*(x/l)**3 - 3*(x/l)**2 + 1

def Nh12(x,l):
    return x**3/l**2 - 2*x**2/l + x

def Nh13(x,l):
    return -2*(x/l)**3 + 3*(x/l)**2

def Nh14(x,l):
    return x**3/l**2 - x**2/l

#Bh1 = [Bh11, Bh12, Bh13, Bh14]

def Bh11(x,l):
    return 6*(x/l)**2 - 6*(x/l)

def Bh12(x,l):
    return 3*(x/l)**2 - 4*x/l + 1

def Bh13(x,l):
    return -6*(x/l)**2 + 6*(x/l)

def Bh14(x,l):
    return 3*(x/l)**2 - 2*x/l

##  Viga sujeita a momento e cortante

$$ EI\frac{\partial^4 y}{\partial x^4} - p(x) = 0 $$


$$ \int_\Omega \left [ EI\frac{\partial^4 y}{\partial x^4} - p(x) \right ] \cdot \omega(x) \; d\Omega = 0 $$

Aplicando Toerema de Green para enfraquecer a equação:

$$ \int_{\Omega} EI\frac{\partial^3 y}{\partial x^3} \frac{\partial \omega(x)}{\partial x} \; d\Omega - \int_{\Gamma} EI\frac{\partial^3 y}{\partial x^3} \omega(x) \; d\Gamma - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Assumindo C.C.'s de Dirichlet:

$$ \int_{\Omega} EI\frac{\partial^3 y}{\partial x^3} \frac{\partial \omega(x)}{\partial x} \; d\Omega - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Repetindo o procedimento:

$$ \int_{\Omega} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial^2 \omega(x)}{\partial x^2} \; d\Omega - \int_{\Gamma} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial \omega(x)}{\partial x} \; d\Gamma - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

$$ \int_{\Omega} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial^2 \omega(x)}{\partial x^2} \; d\Omega - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Discretizando a formulação fraca:

$$ \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} EI\frac{\partial^2 (N_i y_i)}{\partial x^2} \frac{\partial^2 (N_j \omega_j)}{\partial x^2} \; d\Omega - \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} N_k p_k \cdot N_j \omega_j \; d\Omega = 0 $$

$$ \sum_{i}^{\infty} \sum_{j}^{\infty} EI \int_{\Omega} B'_i B'_j \; d\Omega \cdot y_i \omega_j - \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} N_k N_j \; d\Omega \cdot p_k \omega_j = 0 $$

Dividindo a equação por $\omega_j$ e substituindo os somatórios duplos por somatórios de elemento, a equação assume a seguinte forma: 

$$ \sum_{e}^{\infty} EI \int_{\Omega} B'_i B'_j \; d\Omega \cdot y_i - \sum_{e}^{\infty} \int_{\Omega} N_k N_j \; d\Omega \cdot p_k = 0 $$

As integrais das funções de interpolação correspondem a matrizes elementares de rigidez e carga:

$$ K^e = EI \int_{\Omega} B'_i B'_j \; d\Omega \;\;,\;\;\;\; Q^e = \int_{\Omega} N_i N_j \; d\Omega $$

Fazendo a montagem das matrizes globais $K$ e $Q$:

$$ K \cdot y_i - Q \cdot p_k = 0 $$

In [16]:
a = np.array([1,2,3])
b = np.array([1,2,3])

M = np.stack((a,b))
M = M.transpose()
M = M.flatten()
print(M)

[1 1 2 2 3 3]


In [37]:
# parametros da simulacao

## malha
L = 1600.0   # m
nx = 100
ne = nx-1
nGL = 2*nx

## CCs
Yo = 1.0
Yf = 0.0
Thetao = 0.0
Thetaf = 1.0

# propriedades da linha
d = 0.180  # m
t = 0.022   # m
D = d + 2.0*t   # m
A = np.pi*(D**2 - d**2)/4.0   # m^2
I = np.pi*(D**4.0 - d**4.0)/64.0   # m^4
E = 206.0*10.0**9.0   # GPa
rho = 7800.0   # kg/m^3
rho_l = rho*A   # kg/m

# transiente
to = 0.0
tf = 2.0
dt = 0.01
nt = int((tf-to)/dt+1)

# carga
def q(x):
    return 9.81*rho_l*x

# geracao de malha 1D
X = np.linspace(0.0,L,nx)
dbX = np.stack((X,X))
dbX = dbX.transpose()
dbX = dbX.flatten()

IEN_GL = np.zeros( (ne,4),dtype='int' )
for e in range(0,ne-1):
  IEN_GL[e] = [e,e,e+1,e+1]

# lista de indices de condicao de contorno
ccs = [1,-1]
ccY  = [0,nGL - 1]
ccTheta = [1,nGL - 2]

# vetor com valores das condicoes de contorno
Ycc = np.zeros( (nGL),dtype='float' )

for i in ccY:
  Ycc[i] = Yo
  Ycc[i] = Yf

Thetacc = np.zeros( (nGL),dtype='float' )

for i in ccTheta:
  Thetacc[i] = Thetao
  Thetacc[i] = Thetaf

# inicializacao das matrizes globais
K = np.zeros( (ne,nGL),dtype='float' )
Q = np.zeros( (ne,nGL),dtype='float' )

# obtendo matrizes elementares
## funcoes de interpolacao

K11 = lambda x,l : E*I*Bh11(x,l)*Bh11(x,l)
K12 = lambda x,l : E*I*Bh11(x,l)*Bh12(x,l)
K13 = lambda x,l:  E*I*Bh11(x,l)*Bh13(x,l)
K14 = lambda x,l : E*I*Bh11(x,l)*Bh14(x,l)
K21 = lambda x,l : E*I*Bh12(x,l)*Bh11(x,l)
K22 = lambda x,l : E*I*Bh12(x,l)*Bh12(x,l)
K23 = lambda x,l : E*I*Bh12(x,l)*Bh13(x,l)
K24 = lambda x,l : E*I*Bh12(x,l)*Bh14(x,l)
K31 = lambda x,l : E*I*Bh13(x,l)*Bh11(x,l)
K32 = lambda x,l : E*I*Bh13(x,l)*Bh12(x,l)
K33 = lambda x,l : E*I*Bh13(x,l)*Bh13(x,l)
K34 = lambda x,l : E*I*Bh13(x,l)*Bh14(x,l)
K41 = lambda x,l : E*I*Bh14(x,l)*Bh11(x,l)
K42 = lambda x,l : E*I*Bh14(x,l)*Bh12(x,l)
K43 = lambda x,l : E*I*Bh14(x,l)*Bh13(x,l)
K44 = lambda x,l : E*I*Bh14(x,l)*Bh14(x,l)

Q11 = lambda x,l : Nh11(x,l)*Nh11(x,l)
Q12 = lambda x,l : Nh11(x,l)*Nh12(x,l)
Q13 = lambda x,l : Nh11(x,l)*Nh13(x,l)
Q14 = lambda x,l : Nh11(x,l)*Nh14(x,l)
Q21 = lambda x,l : Nh12(x,l)*Nh11(x,l)
Q22 = lambda x,l : Nh12(x,l)*Nh12(x,l)
Q23 = lambda x,l : Nh12(x,l)*Nh13(x,l)
Q24 = lambda x,l : Nh12(x,l)*Nh14(x,l)
Q31 = lambda x,l : Nh13(x,l)*Nh11(x,l)
Q32 = lambda x,l : Nh13(x,l)*Nh12(x,l)
Q33 = lambda x,l : Nh13(x,l)*Nh13(x,l)
Q34 = lambda x,l : Nh13(x,l)*Nh14(x,l)
Q41 = lambda x,l : Nh14(x,l)*Nh11(x,l)
Q42 = lambda x,l : Nh14(x,l)*Nh12(x,l)
Q43 = lambda x,l : Nh14(x,l)*Nh13(x,l)
Q44 = lambda x,l : Nh14(x,l)*Nh14(x,l)

## Funcao de integracao a ser usada
def INTEGRACAO_1(f,e):
     v1,v2,v3,v4 = IEN_GL[e]
     l = dbX[v3] - dbX[v1]
     
     return sp.integrate.quad(f,dbX[v1],dbX[v3],args=(l,))[0]

## Integrando
kelem = np.empty((4,4,ne))
qelem = np.empty((4,4,ne))

for e in range(0,ne):
    Ke = np.array([[INTEGRACAO_1(K11,e),INTEGRACAO_1(K12,e),INTEGRACAO_1(K13,e),INTEGRACAO_1(K14,e)],
                   [INTEGRACAO_1(K21,e),INTEGRACAO_1(K22,e),INTEGRACAO_1(K23,e),INTEGRACAO_1(K24,e)],
                   [INTEGRACAO_1(K31,e),INTEGRACAO_1(K32,e),INTEGRACAO_1(K33,e),INTEGRACAO_1(K34,e)],
                   [INTEGRACAO_1(K41,e),INTEGRACAO_1(K42,e),INTEGRACAO_1(K43,e),INTEGRACAO_1(K44,e)]])     

    Qe = np.array([[INTEGRACAO_1(Q11,e),INTEGRACAO_1(Q12,e),INTEGRACAO_1(Q13,e),INTEGRACAO_1(Q14,e)],
                   [INTEGRACAO_1(Q21,e),INTEGRACAO_1(Q22,e),INTEGRACAO_1(Q23,e),INTEGRACAO_1(Q24,e)],
                   [INTEGRACAO_1(Q31,e),INTEGRACAO_1(Q32,e),INTEGRACAO_1(Q33,e),INTEGRACAO_1(Q34,e)],
                   [INTEGRACAO_1(Q41,e),INTEGRACAO_1(Q42,e),INTEGRACAO_1(Q43,e),INTEGRACAO_1(Q44,e)]]) 

    kelem[:,:,e] = Ke
    qelem[:,:,e] = Qe

for e in range(0,ne):

 for ilocal in range(0,4):
  iglobal = IEN_GL[e,ilocal]
  for jlocal in range(0,4):
   jglobal = IEN_GL[e,jlocal]

   K[iglobal,jglobal] += kelem[ilocal,jlocal,e]
   Q[iglobal,jglobal] += qelem[ilocal,jlocal,e]

# definicao do sistema linear
A = K
b = Q*q(dbX)

# imposicao das c.c.s de Dirichlet na matriz A
for i in ccs:
  for j in ccY:
    A[i,:] = 0.0   # zerar linha
    A[i,j] = 1.0   # incluir 1 na diagonal

for i in ccs:
  for j in ccTheta:
    A[i,:] = 0.0   # zerar linha
    A[i,j] = 1.0   # incluir 1 na diagonal

# imposicao das c.c.s de Dirichlet no vetor b
for i in ccY:
  b[i] = Ycc[i]

for i in ccTheta:
  b[i] = Thetacc[i]

# solucao
cscA = sp.sparse.csc_matrix(A)   # Matrix form that improves sparse.linalg.solve
SOL = sp.sparse.linalg.spsolve(cscA,b)
SOL = np.linalg.solve(A,b)

SOL = SOL.reshape((nGL/2,2))
Y = SOL[:,0]
Theta = SOL[:,1]

plt.scatter(X,Y,s=1.5,color='b')
plt.plot(X,Y,'b-',linewidth=1.0)
plt.show()

C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1740255161.py:20: RuntimeWarning: invalid value encountered in scalar divide
  return 6*(x/l)**2 - 6*(x/l)
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1049194888.py:109: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  return sp.integrate.quad(f,dbX[v1],dbX[v3],args=(l,))[0]
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1740255161.py:23: RuntimeWarning: invalid value encountered in scalar divide
  return 3*(x/l)**2 - 4*x/l + 1
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1740255161.py:26: RuntimeWarning: invalid value encountered in scalar divide
  return -6*(x/l)**2 + 6*(x/l)
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1740255161.py:29: RuntimeWarning: invalid value encountered in scalar divide
  return 3*(x/l)**2 - 2*x/l
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_12172\1740255161.py:6: RuntimeWarning: invalid value encountere

IndexError: index 199 is out of bounds for axis 0 with size 99